## Gaussian mixture check

For the gaussian mix. loss we need to form a Gaussian kernel, when we minimize negative log-likelihood.

In [ ]:
import math
import numpy as np
import tensorflow as tf
import tensorflow_probability as tfp
from tensorflow_probability import distributions as tfd

In [ ]:
#  batch, time, features, mix
mix = tf.random.uniform((2, 1, 2, 6))

## Check probability of kernel

In [ ]:
sample = 0
mu = 0
sig = 0.1

nor = tfd.Normal(loc=mu, scale=sig)
nor.prob(sample)

In [ ]:
constant = 1 / math.sqrt(2 * math.pi)
gaussian_kernel = np.subtract(sample, mu)
gaussian_kernel = tf.square(tf.divide(gaussian_kernel, sig))
gaussian_kernel = - 1/2 * gaussian_kernel
tf.divide(tf.exp(gaussian_kernel), sig) * constant

## Check log probs are the same

In [ ]:
nor.log_prob(sample)

In [ ]:
math.log(nor.prob(sample))

## Incorporating more of the MDN

http://blog.otoro.net/2015/11/24/mixture-density-networks-with-tensorflow/

In [ ]:
inputs = tf.keras.Input(shape=(9,))

# def get_mixture_coef(output):
pi, sigma, mu = tf.split(inputs, 3, 1)

#  put the pi's through a softmax
#  remove the max
max_pi = tf.reduce_max(pi, 1, keepdims=True)
sub_pi = tf.subtract(max_pi, pi)
exp_pi = tf.exp(sub_pi)

out_pi = tf.divide(exp_pi, tf.reduce_sum(exp_pi, 1, keepdims=True))

#  keep the standard deviations positive
sigma = tf.exp(sigma)

model = tf.keras.Model(inputs=inputs, outputs=[out_pi])

In [ ]:
sample = np.random.rand(1,9)
o = model.predict(sample)

#  prints out each output at a time (inputs[0], inputs[1], pi[0], pi[1] etc)\
print(sample)
for out in o:
    for e in out:
        print(e)

In [ ]:
sum(out)

In [ ]:
mean = np.array([10,20]).reshape(1, -1)
sigma = np.array([1,1]).reshape(1, -1)
sample = np.array([9,21]).reshape(1, -1)

alphas = np.array([0.2, 0.8]).reshape(1, -1)  #  already softmaxed etc

def make_gaussian_kernel(mean, sigma, sample):
    constant = 1 / math.sqrt(2 * math.pi)
    gaussian_kernel = np.subtract(sample, mean)
    gaussian_kernel = np.square(np.divide(gaussian_kernel, sigma))
    gaussian_kernel = - 1/2 * gaussian_kernel
    conditional_probabilities = np.divide(np.exp(gaussian_kernel), sigma) * constant
    return conditional_probabilities

In [ ]:
make_gaussian_kernel(mean, sigma, sample)

In [ ]:
#  loss
conditional_probabilities = make_gaussian_kernel(mean, sigma, sample)
print(conditional_probabilities)
loss = np.multiply(conditional_probabilities, alphas)
print(loss)
loss = np.sum(loss, 1, keepdims=True)
print(loss)
loss = -np.log(loss)
print(loss)
loss = np.mean(loss)

print(loss)